<a href="https://www.kaggle.com/code/aryan401/vgg16-fruits?scriptVersionId=142122717" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import torch
import torchvision
from torch import nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader
from sklearn.model_selection import train_test_split
from time import time
from statistics import mean

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
tranform = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train_dataset = torchvision.datasets.ImageFolder(root='../input/fruits/fruits-360_dataset/fruits-360/Training', transform=tranform)
test_dataset = torchvision.datasets.ImageFolder(root='../input/fruits/fruits-360_dataset/fruits-360/Test', transform=tranform)
val_size = int(0.1 * len(train_dataset))
train_size = len(train_dataset) - val_size
train_dataset, val_dataset = torch.utils.data.random_split(
    train_dataset,
    [train_size, val_size]
)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print('Train DataSet len:',len(train_dataset))
print('Valid DataSet len:',len(val_dataset))
print('Test DataSet len:',len(test_dataset))
print('Total DataSet Classes:',len(test_dataset.classes))

Train DataSet len: 60923
Valid DataSet len: 6769
Test DataSet len: 22688
Total DataSet Classes: 131


In [3]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.ConvSet_1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.ConvSet_2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.ConvSet_3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.ConvSet_4 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.ConvSet_5 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.FC_Layers = nn.Sequential(
            nn.Linear(25088, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 131),
        )
        
    def forward(self, x):
        out = self.ConvSet_1(x)
        out = self.ConvSet_2(out)
        out = self.ConvSet_3(out)
        out = self.ConvSet_4(out)
        out = self.ConvSet_5(out)
        out = out.reshape(out.shape[0], -1)
        out = self.FC_Layers(out)
        return out

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VGG16()
model = model.to(device=device)
print(model)
print('Device: ',device)

VGG16(
  (ConvSet_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (ConvSet_2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (ConvSet_3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (

In [5]:
learning_rate = 1e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate)

In [6]:
num_epochs = 5
print(f"Train DataLoader Size: {len(train_dataloader)}")
print(f"Valid DataLoader Size: {len(valid_dataloader)}")
for epoch in range(num_epochs):
    global_time = time()
    local_time_chg = []
    loss_ep = 0
    for images, labels in train_dataloader:
        local_time = time()
        images = images.to(device=device)
        labels = labels.to(device=device) 
        optimizer.zero_grad()
        scores = model(images)
        loss = criterion(scores,labels)
        loss.backward()
        optimizer.step()
        loss_ep += loss.item()
        local_time_chg.append(time()-local_time)
        if len(local_time_chg) % 100 == 0:
            print(f"Step Verified: {len(local_time_chg)}")
        
    with torch.no_grad():
        correct = 0
        total = 0
        print("Starting Validation Test...")
        for images, labels in valid_dataloader:
            images = images.to(device=device)
            labels = labels.to(device=device) 
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy of the Model on the {len(val_dataset)} Valid Images: {100*(correct /total)} %')

    print (f'Epoch [{epoch+1}/{num_epochs}] | Loss: {round(loss_ep/len(train_dataloader), 5)} | Step Time: {mean(local_time_chg)} | Epoch Time: {time() - global_time}')

Train DataLoader Size: 952
Valid DataLoader Size: 106
Step Verified: 100
Step Verified: 200
Step Verified: 300
Step Verified: 400
Step Verified: 500
Step Verified: 600
Step Verified: 700
Step Verified: 800
Step Verified: 900
Starting Validation Test...
Accuracy of the Model on the 6769 Valid Images: 90.50081252769981 %
Epoch [1/5] | Loss: 1.83912 | Step Time: 0.4819644050938742 | Epoch Time: 1162.0153198242188
Step Verified: 100
Step Verified: 200
Step Verified: 300
Step Verified: 400
Step Verified: 500
Step Verified: 600
Step Verified: 700
Step Verified: 800
Step Verified: 900
Starting Validation Test...
Accuracy of the Model on the 6769 Valid Images: 97.68060274782094 %
Epoch [2/5] | Loss: 0.12487 | Step Time: 0.4766195206081166 | Epoch Time: 657.8281271457672
Step Verified: 100
Step Verified: 200
Step Verified: 300
Step Verified: 400
Step Verified: 500
Step Verified: 600
Step Verified: 700
Step Verified: 800
Step Verified: 900
Starting Validation Test...
Accuracy of the Model on the

In [7]:
torch.save(model.state_dict(), "/kaggle/working/VGG16_Fruit.pt")
model = VGG16()

In [8]:
model.load_state_dict(torch.load("/kaggle/working/VGG16_Fruit.pt"))
model = model.to(device=device)
model.eval()

VGG16(
  (ConvSet_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (ConvSet_2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (ConvSet_3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (

In [9]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device=device)
        labels = labels.to(device=device) 
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the Model on the {} Test Images: {}%'.format(len(test_dataset), 100*correct /total))

Accuracy of the Model on the 22688 Test Images: 89.27186177715092%
